## Setup

In [1]:
import pandas as pd
import numpy as np

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

## In memory data

For any small CSV dataset the simplest way to train a TensorFlow model on it is to load it into memory as a pandas Dataframe or a NumPy array. 


A relatively simple example is the [abalone dataset](https://archive.ics.uci.edu/ml/datasets/abalone). 

* The dataset is small. 
* All the input features are all limited-range floating point values. 

Here is how to download the data into a [Pandas `DataFrame`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html):

In [2]:
abalone_train = pd.read_csv(
    "https://storage.googleapis.com/download.tensorflow.org/data/abalone_train.csv",
    names=["Length", "Diameter", "Height", "Whole weight", "Shucked weight",
           "Viscera weight", "Shell weight", "Age"])

abalone_train.head()

,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Age
0,0.435,0.335,0.110,0.334,0.1355,0.0775,0.0965,7
1,0.585,0.450,0.125,0.874,0.3545,0.2075,0.2250,6
2,0.655,0.510,0.160,1.092,0.3960,0.2825,0.3700,14
3,0.545,0.425,0.125,0.768,0.2940,0.1495,0.2600,16
4,0.545,0.420,0.130,0.879,0.3740,0.1695,0.2300,13


The dataset contains a set of measurements of abalone, a type of sea snail.

The nominal task for this dataset is to predict the age from the other measurements, so separate the features and labels for training:


In [3]:
abalone_features = abalone_train.copy()
abalone_labels = abalone_features.pop('Age')

For this dataset you will treat all features identically. Pack the features into a single NumPy array.:

In [4]:
abalone_features = np.array(abalone_features)
abalone_features

array([[0.435, 0.335, 0.11 , ..., 0.136, 0.077, 0.097],
       [0.585, 0.45 , 0.125, ..., 0.354, 0.207, 0.225],
       [0.655, 0.51 , 0.16 , ..., 0.396, 0.282, 0.37 ],
       ...,
       [0.53 , 0.42 , 0.13 , ..., 0.374, 0.167, 0.249],
       [0.395, 0.315, 0.105, ..., 0.118, 0.091, 0.119],
       [0.45 , 0.355, 0.12 , ..., 0.115, 0.067, 0.16 ]])

Next make a regression model predict the age. Since there is only a single input tensor, a `keras.Sequential` model is sufficient here.

In [5]:
abalone_model = tf.keras.Sequential([
  layers.Dense(64),
  layers.Dense(1)
])

abalone_model.compile(loss = tf.losses.MeanSquaredError(),
                      optimizer = tf.optimizers.Adam())

To train that model, pass the features and labels to `Model.fit`:

In [6]:
abalone_model.fit(abalone_features, abalone_labels, epochs=10)

Epoch 1/10
104/104 [==============================] - 0s 805us/step - loss: 57.6522
Epoch 2/10
104/104 [==============================] - 0s 1ms/step - loss: 10.9227
Epoch 3/10
104/104 [==============================] - 0s 844us/step - loss: 8.4763
Epoch 4/10
104/104 [==============================] - 0s 1ms/step - loss: 7.9622
Epoch 5/10
104/104 [==============================] - 0s 959us/step - loss: 7.5199
Epoch 6/10
104/104 [==============================] - 0s 949us/step - loss: 7.1713
Epoch 7/10
104/104 [==============================] - 0s 1ms/step - loss: 6.8964
Epoch 8/10
104/104 [==============================] - 0s 854us/step - loss: 6.7148
Epoch 9/10
104/104 [==============================] - 0s 911us/step - loss: 6.5710
Epoch 10/10
104/104 [==============================] - 0s 804us/step - loss: 6.4677


You have just seen the most basic way to train a model using CSV data. Next, you will learn how to apply preprocessing to normalize numeric columns.

## Basic preprocessing

It's good practice to normalize the inputs to your model. The `experimental.preprocessing` layers provide a convenient way to build this normalization into your model. 

The layer will precompute the mean and variance of each column, and use these to normalize the data.

First you create the layer:

In [7]:
normalize = preprocessing.Normalization()

Then you use the `Normalization.adapt()` method to adapt the normalization layer to your data.

Note: Only use your training data to `.adapt()` preprocessing layers. Do not use your validation or test data.

In [8]:
normalize.adapt(abalone_features)

Then use the normalization layer in your model:

In [9]:
norm_abalone_model = tf.keras.Sequential([
  normalize,
  layers.Dense(64),
  layers.Dense(1)
])

norm_abalone_model.compile(loss = tf.losses.MeanSquaredError(),
                           optimizer = tf.optimizers.Adam())

norm_abalone_model.fit(abalone_features, abalone_labels, epochs=10)

Epoch 1/10
104/104 [==============================] - 0s 759us/step - loss: 91.9988
Epoch 2/10
104/104 [==============================] - 0s 969us/step - loss: 53.4315
Epoch 3/10
104/104 [==============================] - 0s 892us/step - loss: 17.3280
Epoch 4/10
104/104 [==============================] - 0s 921us/step - loss: 6.1479
Epoch 5/10
104/104 [==============================] - 0s 1ms/step - loss: 5.1279
Epoch 6/10
104/104 [==============================] - 0s 1ms/step - loss: 5.0363
Epoch 7/10
104/104 [==============================] - 0s 1ms/step - loss: 4.9968
Epoch 8/10
104/104 [==============================] - 0s 1ms/step - loss: 4.9591
Epoch 9/10
104/104 [==============================] - 0s 969us/step - loss: 4.9582
Epoch 10/10
104/104 [==============================] - 0s 969us/step - loss: 4.9430
